# Your Title Here

This template contains some of the code you may need for your Hackathon project. Some of the other code, you will have to figure out yourself -- it depends on what model you use, and what the goal of your project is.

> **Don't forget to change the title of this notebook to something catchy, and describe your project briefly here!**
> 
> Team Members: [ ]
> 
> Model Used: [ ]
> 
> Peripherals Used: [ ]
> 
> Goal of the Project: [ ]


## Imports

Most likely, you will need things from each of those imports.

In [1]:
from pynq_dpu import DpuOverlay   # Overlay for the FPGA
from pynq_peripherals import *    # Peripherals
import numpy as np                # Numerical library for matrix calculations
import cv2                        # OpenCV image processing library
import matplotlib.pyplot as plt   # Plotting Library
import IPython.display            # Display Widget in Jupyter Notebook


## Loading the model

This will prepare your FPGA with a DPU overlay, and then load your model onto that overlay.

In [2]:
xmodelPath = ""  # <- Put your model here
overlay = DpuOverlay("dpu.bit")
overlay.load_model(xmodelPath)
dpu = overlay.runner

> **Note:** If you run into a "Device or resource busy" error, quit all your Jupyter notebook kernels and restart this one. Then, try again. You can quit notebook kernels by clicking Kernel > Shutdown and restart by clicking Kernel > Restart.

Let's see what the input and output dimensions of this model are:

In [3]:
inputTensors  = dpu.get_input_tensors()
outputTensors = dpu.get_output_tensors()

shapeIn    = tuple(inputTensors[0].dims)
shapeOut   = tuple(outputTensors[0].dims)
outputSize = int(outputTensors[0].get_data_size() / shapeIn[0])

print("shapeIn:    {}".format(shapeIn))
print("shapeOut:   {}".format(shapeOut))
print("outputSize: {}".format(outputSize))

shapeIn:    (1, 368, 368, 3)
shapeOut:   (1, 46, 46, 26)
outputSize: 55016


## Input Preprocessing

At this point, you will probably need to get some input data from somewhere, and then preprocess it to feed it into your model.

> Briefly describe here what you had to do for preprocessing.

> **Note:** Make sure that the input has the correct shape (`shapeIn` from above) after your preprocessing!

In [4]:
preprocessedInput = np.random.rand(*shapeIn)  # <- add your work here

## Run the model

We will put our preprocessed input from above in an inputData array, and prepare space for the results with an empty outputData array. When the model is finished running, it will put its results in the outputData array.

In [5]:
modelDtype   = np.float32
outputData   = [np.zeros(shapeOut, dtype=modelDtype, order="C")]
inputData    = [np.zeros(shapeIn,  dtype=modelDtype, order="C")]
inputData[0] = preprocessedInput.reshape(shapeIn).astype(modelDtype, order="C")
job_id = dpu.execute_async(inputData, outputData)
dpu.wait(job_id)

0

## Post-process the output

At this point, it is up to you to make sense of the output of the model.

It will likely give you a matrix of numbers. To make sense of these numbers, you may need to do some post processing, like summing them up.

> Describe what post-processing you had to do.

In [7]:
myOutput = outputData[0]

# We are going to print some statistics about the output as an example.
# Feel free to remove everything below.
print("Output")
print("Shape:   {}".format(myOutput.shape))
print("Minimum: {}".format(np.min(myOutput)))
print("Maximum: {}".format(np.max(myOutput)))
print("Mean:    {}".format(np.mean(myOutput)))
print("Median:  {}".format(np.median(myOutput)))
print("Stddev:  {}".format(np.std(myOutput)))

Output
Shape:   (1, 46, 46, 26)
Minimum: -0.0078125
Maximum: 0.0078125
Mean:    8.577050175517797e-05
Median:  0.0
Stddev:  0.0008302718051709235


## Display your results

After post-processing, show your results!

## Teardown

At the end, it is good to destroy your DPU objects. This way, you won't have any issues using your KRIA from another notebook later.

In [8]:
del dpu
del overlay